In [ ]:
import pandas as pd
#Cargue del Dataset y Elimino columnas que no me aportan al modelo 
Datos = pd.read_excel('mask_info_cli_ros_gmr.xlsx')
Datos.drop(['Unnamed: 0'], axis=1, inplace=True)
display(Datos)

**Se Identifico un Registro mal Ingresado tenia el texto (CAPTURADO POR NARCOTRÁFICO CASO 1086954                                                                        ") se le puso el mismo valor que tenia la columna frecuencia_total_anual_transada__efectivo_**

In [ ]:
Datos['monto_total_anual_transado__efectivo_'] = Datos['monto_total_anual_transado__efectivo_'].replace({'CAPTURADO POR NARCOTRÁFICO CASO 1086954                                                                        "': 42940000})
Datos["monto_total_anual_transado__efectivo_"] = pd.to_numeric(Datos["monto_total_anual_transado__efectivo_"], errors='coerce')

## **Reporte Dataset Inicial**

In [ ]:
#pip install ydata-profiling

In [ ]:
from ydata_profiling import ProfileReport
from ydata_profiling.utils.cache import cache_file
Reporte = ProfileReport(Datos, title="Reporte Datos Entregados", minimal = True)
Reporte.to_notebook_iframe()
Reporte.to_file("ReporteDataset.html")

In [ ]:
#Elimino Todas las Columnas que no me aportan al modelo 
Datos.drop(['nombre','num_caso','cod_ciiu','cod_ocup','cod_ciudad_dirp','cod_pais_nacim','pais_origen_recursos','pais_residencia','cod_categ_lc','riesgo_cliente__ric_','cod_subcateg_lc','cod_nivel_cat'], axis=1, inplace=True)
Datos.drop(['act_econom','desc_categ','motivo_ingreso_a_listas_de_control','ros'], axis=1, inplace=True)
Datos.drop(['cod_tipo_doc', 'f_vinc', 'f_ingreso_lc'], axis=1, inplace=True)
Datos.shape

In [ ]:
#Identifico variables categoricas y las Numericas
num_cols = Datos.select_dtypes(exclude=['object']).columns
cat_cols = Datos.select_dtypes(include=['object']).columns

In [ ]:
from sklearn.impute import SimpleImputer
#Imputo Columnas Numericas
imputer = SimpleImputer(strategy='constant', fill_value=0)
NumImp = pd.DataFrame(imputer.fit_transform(Datos[num_cols]), columns=Datos[num_cols].columns)

In [ ]:
#Imputo Columnas Categoricas
imputer = SimpleImputer(strategy='constant', fill_value='Sin Informacion')
CatImp = pd.DataFrame(imputer.fit_transform(Datos[cat_cols]), columns=Datos[cat_cols].columns)

In [ ]:
#Concateno el Dataframe con columnas ya Imputadas
Datos = pd.concat([CatImp, NumImp], axis=1)

In [ ]:
#Para Disminuir la Dimencion de los valores Dividi en rangos estas 6 Columnas
columnas = ['ing_mes','egresos_mes','monto_total_anual_transado__efectivo_','monto_total_anual_transado__operaciones_internacionales_','frecuencia_total_anual_transada__operaciones_internacionales_','frecuencia_total_anual_transada__efectivo_']
Convertir = Datos[columnas]

In [ ]:
# Función para convertir en Rangos 
def asignar_rangos(df, columnas):
    def dividir_en_rangos(col):
        valores_unicos = sorted(col.dropna().unique()) # Obtener valores únicos y ordenarlos
        total_valores = len(valores_unicos)
        if total_valores < 5:
            return col.apply(lambda x: "Rango menor de 5" if pd.notna(x) else x)
        block_size = total_valores // 5 # Definir el tamaño de cada bloque
        remainder = total_valores % 5
        rangos = []
        start = 0
        for i in range(5):
            end = start + block_size + (1 if i < remainder else 0)
            if end > total_valores:
                end = total_valores
            min_val = valores_unicos[start]
            max_val = valores_unicos[end - 1]
            rangos.append((min_val, max_val))
            start = end
        def asignar_rango(valor):
            if pd.isna(valor):
                return valor
            for min_val, max_val in rangos:
                if min_val <= valor <= max_val:
                    return f"Rango entre {min_val} y {max_val}"
        return col.apply(asignar_rango)
    for columna in columnas:
        if columna in df.columns:
            df.loc[:, columna] = dividir_en_rangos(df[columna]) 
    return df
Rangos = asignar_rangos(Convertir.copy(), columnas)

In [ ]:
#Elimine las columnas indicadas y adicione las nuevas ya con los rangos
Datos.drop(columns=columnas, inplace=True)
Datos = pd.concat([Datos, Rangos], axis=1)

In [ ]:
#Genero una Copia de los Datos Ya Preparados
DatOrig = Datos.copy()

**Preparación para K-prototypes**

In [ ]:
#Identifico variables categoricas y las Numericas
num_cols = Datos.select_dtypes(exclude=['object']).columns
cat_cols = Datos.select_dtypes(include=['object']).columns

**Aplicar K-prototypes**

In [ ]:
#Conversión de DataFrame a matriz de Numpy
data_matrix = Datos.values

In [ ]:
#Asignacion Manual del Total de Cluster
num_clusters=5

In [ ]:
from kmodes.kprototypes import KPrototypes
kproto = KPrototypes(n_clusters=num_clusters,init='Huang',n_init=10,max_iter=150,verbose=1)
clusters = kproto.fit_predict(data_matrix, categorical=[Datos.columns.get_loc(col) for col in cat_cols])

In [ ]:
# Añadir la información de clusters al DataFrame original
DatOrig['cluster'] = clusters

In [ ]:
#Agrupo por cluster y cuento cuantos registros asigno a cada cluster
DatOrig.groupby('cluster').cluster.count().sort_values(ascending=False)

In [ ]:
DataOriginal = DatOrig.copy()

In [ ]:
# Obtención de los clusters únicos
NumeroCluster = DatOrig["cluster"].unique()

In [ ]:
# Lista para almacenar los nombres de los datasets segmentados
nombres_variables = []
for cluster in NumeroCluster:
    df_segmentado = DatOrig[DatOrig["cluster"] == cluster]
    Ncluster = f"Cluster_0{cluster}"
    globals()[Ncluster] = df_segmentado
    nombres_variables.append(Ncluster)
print("Datasets segmentados creados exitosamente.")

In [ ]:
todoscluster = []
for cluster in NumeroCluster:
    Ncluster = f"Cluster_0{cluster}"
    todoscluster.append(Ncluster)

Adicional tambien Retirare para generar Reglas de Asociacion las columnas:
* doc
* cluster

Tambien Adicionare las Columnas que bancolombia me diga que no son relevantes

In [ ]:
ColumnEli =['doc','cluster']

In [ ]:
def depura_columnas(nombres_cluster, columnas_eliminar):
    for nombre in nombres_cluster:
        # Usamos globals() para acceder a la variable con el nombre dinámicamente
        if nombre in globals():
            df = globals()[nombre]
            if isinstance(df, pd.DataFrame):
                # Creamos una copia del DataFrame con las columnas eliminadas y la reasignamos a la variable global
                globals()[nombre] = df.drop(columns=columnas_eliminar, axis=1)

In [ ]:
# Llamada a la función
depura_columnas(todoscluster, ColumnEli)

# **Reporte Analisis Caracteristica Cluster Generado**

Este Genera un reporte en texto con el analisis del contenido por cada Cluster

In [ ]:
def analizar_columnas(df):
    resultados = []
    for columna in df.columns:
        value_counts = df[columna].value_counts()
        ItmasAlto = value_counts.idxmax()
        ValmasAlto = value_counts.max()
        total_items = value_counts.sum()
        PormasAlto = (ValmasAlto / total_items) * 100
        resultado = (f"De la columna '{columna}': El {PormasAlto:.2f}% de los Registros es '{ItmasAlto}' "
                     f"Con un total de {ValmasAlto} Items")
        resultados.append(resultado)
    return resultados

In [ ]:
def guardar_resultados_en_archivo(lista_nombres_df, archivo_salida):
    with open(archivo_salida, 'w') as file:
        for nombre_df in lista_nombres_df:
            df = globals().get(nombre_df)
            if df is None or not isinstance(df, pd.DataFrame):
                print(f"El DataFrame '{nombre_df}' no existe o no es un DataFrame. Se omite.")
                continue
            file.write(f"Resultados del Cluster '{nombre_df}':\n")
            resultados = analizar_columnas(df)
            for resultado in resultados:
                file.write(resultado + "\n")
            file.write("-" * 50 + "\n")

In [ ]:
# Llamar a la función para guardar los resultados en un archivo de texto
guardar_resultados_en_archivo(todoscluster, 'Caracterizacion_clusters.txt')

# **Reporte de Reglas de Asociacion**

In [112]:
from sklearn.preprocessing import OneHotEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules
from joblib import Parallel, delayed
from tabulate import tabulate

In [113]:
# Función para procesar cada segmento utilizando FP-Growth
def procesar_segmento(segmento, min_support=0.01, min_threshold=0.9):
    conjuntos_frecuentes = fpgrowth(segmento, min_support=min_support, use_colnames=True)
    if conjuntos_frecuentes.shape[0] > 0:
        reglas_asociacion = association_rules(conjuntos_frecuentes, metric='confidence', min_threshold=min_threshold)
        reglas_con_mas_caracteristicas = reglas_asociacion[reglas_asociacion['antecedents'].apply(lambda x: len(x) >= 2)]
        return reglas_con_mas_caracteristicas
    return None

In [114]:
for cluster_name in todoscluster:
    Cluster = globals()[cluster_name]
    # Binarizar el dataset con OneHotEncoder
    encoder = OneHotEncoder(sparse_output=False, drop='first')
    datos_binarizados = encoder.fit_transform(Cluster)
    Cluster_binarizado = pd.DataFrame(datos_binarizados, columns=encoder.get_feature_names_out(Cluster.columns))
    tamaño_segmento = 100 # Dividir las columnas en segmentos de tamaño especificado (ajustado para one-hot encoding)
    segmentos_columnas = [Cluster_binarizado.iloc[:, i:i+tamaño_segmento] for i in range(0, Cluster_binarizado.shape[1], tamaño_segmento)]
    # Procesar los segmentos en paralelo
    todas_reglas_asociacion = Parallel(n_jobs=-1)(delayed(procesar_segmento)(segmento, min_support=0.1, min_threshold=0.6) for segmento in segmentos_columnas)
    # Filtrar reglas vacías
    todas_reglas_asociacion = [reglas for reglas in todas_reglas_asociacion if reglas is not None]
    # Si se generaron reglas, concatenarlas y aplicar filtros adicionales
    if todas_reglas_asociacion:
        reglas_asociacion_total = pd.concat(todas_reglas_asociacion, ignore_index=True)
        # Aplicar filtros para reducir el número de reglas
        reglas_filtradas = reglas_asociacion_total[
            (reglas_asociacion_total['support'] >= 0.05) &
            (reglas_asociacion_total['confidence'] >= 0.8) &
            (reglas_asociacion_total['lift'] >= 1.5)
        ]
        # Ordenar las reglas por confianza y seleccionar las más fuertes
        reglas_filtradas = reglas_filtradas.sort_values(by='confidence', ascending=False)#.head(100)
        # Redondear valores para mejor presentación
        reglas_filtradas = reglas_filtradas.round(decimals=3)
        reglas_filtradas['antecedent support'] = reglas_filtradas['antecedent support'].apply(lambda x: '{:,.0f}'.format(x))
        reglas_filtradas['consequent support'] = reglas_filtradas['consequent support'].apply(lambda x: '{:,.0f}'.format(x))
        reglas_filtradas['support'] = reglas_filtradas['support'].apply(lambda x: '{:,.0f}'.format(x))
        reglas_filtradas['lift'] = reglas_filtradas['lift'].apply(lambda x: '{:,.3f}'.format(x)) 
        # Definir una función para formatear los frozensets
        def format_frozenset(fset):
            elements = [elem.strip() for elem in str(fset).strip("frozenset({})").split(",")]
            return ", ".join(elements)
        # Formatear las columnas antecedents y consequents para eliminar "frozenset({...})"
        reglas_filtradas['antecedents'] = reglas_filtradas['antecedents'].apply(lambda x: format_frozenset(x))
        reglas_filtradas['consequents'] = reglas_filtradas['consequents'].apply(lambda x: format_frozenset(x))
        # Exportar el dataframe a un archivo CSV
        reglas_filtradas.to_csv(f'Reglas_Asociacion_{cluster_name}.csv', index=False)